In [1]:
# load packages
import numpy as np
import pandas as pd
import seaborn as sns
import time
import matplotlib.pyplot as plt

from sklearn.ensemble import RandomForestRegressor,RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

import torch
import esm
from tqdm import tqdm
import pickle

from torchinfo import summary

from sklearn.metrics import f1_score, roc_auc_score, roc_curve, auc, precision_recall_curve
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, MinMaxScaler

import sys
sys.path.append('./../../src/')


from utils import *
from TDPredictor_CNN import *
from MHCCBM import *

import itertools
import json

In [2]:
hidden_channels_ls = [[1024], [512], [256], [128], [64]]
epochs = 200
batch_size_ls = [8,16,32]
lr_ls = [1e-3, 1e-5]
ksize_ls = [3, 5]
dropout_p_ls = [0, 0.1]

combinations = [i for i in itertools.product(hidden_channels_ls, ksize_ls, batch_size_ls, lr_ls, dropout_p_ls)]
ncombinations = len(combinations)

# for i, (hc, ks, bs, lr, dp) in enumerate(itertools.product(hidden_channels_ls, ksize_ls, batch_size_ls, lr_ls, dropout_p_ls)):
#     print(i, (hc, ks, bs, lr, dp))
    
#     config_dict = { "project" : "MHCCBM",
#                     "name" : "CNN_"+str(i), 
#                     "config" : {
#                         "hidden_channels" : hc,
#                         "epochs" : epochs,
#                         "kernel_size": ks,
#                         "pool_kernel_size":ks,
#                         "classes" : 2,
#                         "seed": 42,
#                         "batch_size" : bs,
#                         "lr" : lr,
#                         "dataset" : "TD bashirova MFI data",
#                         "dropout_p" : dp,
#                         "architecture" : "CNN"}
#                     }
#     with open('./config/hyperparameter_tuning/CNN/config_'+str(i)+'.json','w') as f:
#         json.dump(config_dict,f)


# config_dict = { "project" : "MHCCBM",
#                     "name" : "CNN_"+str(120), 
#                     "config" : {
#                         "hidden_channels" : [256,256],
#                         "epochs" : 200,
#                         "kernel_size": 3,
#                         "pool_kernel_size":3,
#                         "classes" : 2,
#                         "seed": 42,
#                         "batch_size" : 16,
#                         "lr" : 1e-05,
#                         "dataset" : "TD bashirova MFI data",
#                         "dropout_p" : 0.1,
#                         "architecture" : "CNN"}
#                     }
# with open('./config/hyperparameter_tuning/CNN/config_'+str(120)+'.json','w') as f:
#     json.dump(config_dict,f)

In [3]:
result_df = []

for idx, (hc, ks, bs, lr, dp) in enumerate(itertools.product(hidden_channels_ls, ksize_ls, batch_size_ls, lr_ls, dropout_p_ls)):
    if bs != 8:  
        result_ = pd.read_csv('./results/hyperparameter_tuning_result/CNN/CNN_'+str(idx)+'.csv')
        result_['idx'] = [idx]*len(result_)
        result_df = result_df + [result_]
        
    
result_df = pd.concat(result_df)
result_df

,Unnamed: 0,time_elapsed,epoch,fold,f1_train,auroc_train,auprc_train,f1_test,auroc_test,auprc_test,idx
0,0,209.227166,68.0,0,0.927564,0.992593,0.998324,0.836601,0.875000,0.985243,4
1,0,309.909894,87.0,1,0.962938,1.000000,1.000000,0.680556,1.000000,1.000000,4
2,0,265.861054,71.0,2,0.927973,0.995536,0.999038,0.777778,0.928571,0.980867,4
3,0,420.934111,121.0,3,0.916635,0.995536,0.999049,0.622222,0.571429,0.883787,4
4,0,631.299865,175.0,4,0.939451,0.996652,0.999295,0.874074,1.000000,1.000000,4
...,...,...,...,...,...,...,...,...,...,...,...
5,0,12.775485,54.0,5,0.860851,0.771205,0.902363,0.555556,0.500000,0.851644,119
6,0,14.309277,58.0,6,0.868998,0.867188,0.963553,0.680556,0.500000,0.692092,119
7,0,32.129847,132.0,7,0.862457,0.923958,0.981569,0.673077,0.428571,0.900425,119
8,0,49.889982,NaN,8,0.889887,0.943750,0.986998,0.891026,1.000000,1.000000,119


In [5]:
result_df.groupby('idx').mean().sort_values('auroc_test')

,Unnamed: 0,time_elapsed,epoch,fold,f1_train,auroc_train,auprc_train,f1_test,auroc_test,auprc_test
idx,,,,,,,,,,
22,0.0,362.612069,55.571429,4.5,0.443268,0.749744,0.911187,0.435210,0.457143,0.809861
45,0.0,144.800521,59.000000,4.5,0.673207,0.831829,0.948375,0.645303,0.623214,0.883360
23,0.0,521.508116,58.000000,4.5,0.658844,0.805188,0.937105,0.563925,0.635714,0.862868
10,0.0,430.490794,53.000000,4.5,0.656210,0.867436,0.957226,0.655167,0.642857,0.868844
106,0.0,27.697171,71.400000,4.5,0.799613,0.880790,0.965352,0.629708,0.664286,0.866088
...,...,...,...,...,...,...,...,...,...,...
52,0.0,97.362312,104.111111,4.5,0.856064,0.996749,0.999299,0.662208,0.942857,0.984552
101,0.0,25.028382,97.666667,4.5,0.969545,0.999219,0.999832,0.765332,0.942857,0.984552
40,0.0,242.369043,116.800000,4.5,0.946620,0.997254,0.999403,0.768917,0.950000,0.985760


In [6]:
result_df[result_df['idx']==80]

,Unnamed: 0,time_elapsed,epoch,fold,f1_train,auroc_train,auprc_train,f1_test,auroc_test,auprc_test,idx
0,0,53.031745,116.0,0,0.914812,0.975661,0.994263,0.777778,0.875000,0.985243,80
1,0,57.680044,138.0,1,0.926644,0.987723,0.997297,0.680556,1.000000,1.000000,80
2,0,51.062383,122.0,2,0.939451,0.986607,0.997119,0.874074,0.928571,0.980867,80
3,0,49.741168,121.0,3,0.926644,0.987723,0.997381,0.680556,0.857143,0.966270,80
4,0,56.741831,135.0,4,0.987350,0.996652,0.999295,0.874074,1.000000,1.000000,80
5,0,83.355085,NaN,5,0.927973,0.993304,0.998561,0.777778,0.928571,0.980867,80
6,0,82.762645,NaN,6,0.894285,0.995536,0.999008,1.000000,1.000000,1.000000,80
7,0,82.962717,NaN,7,0.963296,1.000000,1.000000,0.686364,1.000000,1.000000,80
8,0,83.272877,NaN,8,0.951529,0.995833,0.999074,1.000000,1.000000,1.000000,80
9,0,68.894672,164.0,9,0.963296,0.998958,0.999758,0.791667,1.000000,1.000000,80


In [21]:
combinations[80], combinations[80], combinations[80]

(([128], 3, 32, 0.001, 0), ([128], 3, 32, 0.001, 0), ([128], 3, 32, 0.001, 0))

In [9]:
pd.read_csv('./results/hyperparameter_tuning_result/CNN/CNN_'+str(121)+'.csv').mean()

Unnamed: 0        0.000000
time_elapsed    653.995264
epoch           131.500000
fold              4.500000
f1_train          0.794804
auroc_train       0.920748
auprc_train       0.907261
f1_test           0.697193
auroc_test        0.780000
auprc_test        0.743254
dtype: float64